# Minesweeper Neural Network

What this essentially does, is it looks at the 7x7 surroundings of a tile. By training on many, many possible cases, it eventually learns the probabilities that a tile is a mine given its 7x7 surroundings.

**Setting up interface to Minesweeper game**

In [1]:
import jpype
import jpype.imports
from jpype.types import *
import numpy as np

print(jpype.getDefaultJVMPath())

jpype.startJVM(jpype.getDefaultJVMPath(),"-ea", "-Djava.class.path=%s" % ('/Users/apimienta/Documents/Minesweeper AI/train'))

MinesweeperInterface = jpype.JClass("MinesweeperInterface")
MinesweeperGame = jpype.JClass("MinesweeperInterface$MinesweeperGame")

/Library/Java/JavaVirtualMachines/jdk1.8.0_102.jdk/Contents/Home/jre/lib/jli/libjli.dylib


In [2]:
"""Generate game and return a tuple of (game, interface)"""
def generate_new_game():
    new_interface = MinesweeperInterface()
    new_game = MinesweeperGame(new_interface)
    
    return (new_game, new_interface)

"""dels the game for garbage collection"""
def destroy_game(interface, game):
    del interface
    del game
    
"""Get board of game as numpy array"""
def get_np_board(game):
    j_board = game.getPlayerBoard()
    return np.pad(np.array([[cell for cell in row] for row in j_board]), ((3, 3), (3, 3)), 'constant', constant_values=-3) # 0-pad as numpy array

"""Click at index [x][y] and return whether or not it was a valid click"""
def click_game(game, x, y):
    return game.click(x, y)
    
new_game, new_interface = generate_new_game()
print(click_game(new_game, 3, 3))
print(get_np_board(new_game).shape)
destroy_game(new_game, new_interface)

True
(22, 22)


In [ ]:
jpype.shutdownJVM()

**Set up model**

In [3]:
from keras import layers
from keras import models
from keras import Model
from keras.optimizers import Adam

def build_model():
    inputs = layers.Input(shape=(49,))
    
    x = layers.Dense(64, activation='relu')(inputs)
    x = layers.Dense(8, activation='relu')(x)
    x = layers.Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=inputs, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    
    return model

In [4]:
build_model().summary()

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 49)]              0         
_________________________________________________________________
dense (Dense)                (None, 64)                3200      
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 520       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 3,729
Trainable params: 3,729
Non-trainable params: 0
_________________________________________________________________


In [9]:
episodes = 1000

def next_to_known(state, x, y):
    x += 3
    y += 3
    for x_ in range(x-1, x+2):
        for y_ in range(y-1, y+2):
            if state[y_][x_] > -1: return True
    return False
    
def get_surroundings(state, x, y):
    x += 3
    y += 3
    return state[y-3:y+4, x-3:x+4].flatten()

"""
Generator for training data
"""
def gen_train_data(model, num_episodes, verbose=False):
    # Gathering data
    for i in range(1, episodes+1):
        game, interface = generate_new_game()
        if verbose and i%1 == 0:
            print("%dth episode" % (i))
        # Set an initial condition
        while not click_game(game, np.random.randint(0, 16), np.random.randint(0, 16)):
            two_plus_two = 4
            
        while not game.isSolved():
            state = get_np_board(game)
            train_queue = [(model.predict(np.array([get_surroundings(state, x, y)]))[0][0], 0 if click_game(game, x, y) else 1, get_surroundings(state, x, y), x, y) for y in range(16) for x in range(16) if state[y+3][x+3] == -1 and next_to_known(state, x, y)]
            for item in train_queue:
                if item[1] == 1:
                    game.setFlag(item[-2], item[-1], 1)
                yield item
            
            if np.array_equal(get_np_board(game), state): break
            
                
        destroy_game(game, interface)

model = build_model()

train_generator = gen_train_data(model, episodes, verbose=True)
history = []

i = 1
while True:
    try:
        prediction, y, x, tileX, tileY = next(train_generator)
        history.append([y, prediction])
        model.fit(np.array([x]), np.array([y]), epochs=1, verbose=False)
        
        if i%1000 == 0:
            correct = 0
            for f in range(1, 1000):
                if abs(history[-f][0] - history[-f][1]) < 0.1:
                    correct += 1
                    
            print("%d correct" % (correct))
        i += 1
    except StopIteration:
        break

1th episode
2th episode
3th episode
4th episode
466 correct
5th episode
6th episode
7th episode
8th episode
544 correct
9th episode
10th episode
11th episode
661 correct
12th episode
13th episode
14th episode
700 correct
15th episode
16th episode
17th episode
705 correct
18th episode
19th episode
20th episode
21th episode
683 correct
22th episode
23th episode
24th episode
676 correct
25th episode
26th episode
27th episode
675 correct
28th episode
29th episode
30th episode
693 correct
31th episode
32th episode
33th episode
34th episode
677 correct
35th episode
36th episode
37th episode
647 correct
38th episode
39th episode
40th episode
41th episode
601 correct
42th episode
43th episode
44th episode
45th episode
632 correct
46th episode
47th episode
48th episode
679 correct
49th episode
50th episode
51th episode
735 correct
52th episode
53th episode
54th episode
55th episode
56th episode
689 correct
57th episode
58th episode
59th episode
642 correct
60th episode
61th episode
62th episode

In [11]:
sur = np.array([[0, 0, 0, 0, 0, 0, 0],
                [2, 2, 1, 1, 0, 0, 0],
                [-1, -1, -2, 2, 1, 1, 0],
                [-1, -1, -1, -1, -2, 2, 0],
                [-1, -1, -1, -1, -2, 2, 0],
                [-1, -1, -1, -1, -1, 2, 1],
                [-1, -1, -1, -1, -1, -1, -1]]).flatten()

print(sur)

model.predict(np.array([sur]))

[ 0  0  0  0  0  0  0  2  2  1  1  0  0  0 -1 -1 -2  2  1  1  0 -1 -1 -1
 -1 -2  2  0 -1 -1 -1 -1 -2  2  0 -1 -1 -1 -1 -1  2  1 -1 -1 -1 -1 -1 -1
 -1]


array([[0.0620447]], dtype=float32)